## Importing modules and packages

In [1]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
# import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
# import gr_process
import roadmatch

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

## Input parameters

In [2]:
trailname = 'gr131' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
delta_roads = 0.010 # Tolerance around bounding box per trail section [deg]

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg
buffersize = 0.00015 # buffer used when merging landuse places
tol_area = 15.0e-6 # buffer used to define which merged landuse places we consider

# Settings for places2processed
tol_d        = 0.75 # Consider a segment developed if it lies closer than tol_d to a developed area
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

## Loading GPX file

In [3]:
gpx = gr_utils.get_gpx(trailname)
gpx_coords = gr_mapmatch.trail_to_coords(gpx) # Convert the points into a list of [lat, lon] pairs

Loading trail points from <data_input/gr131.gpx>...
Finished loading.


## Matching GPX points to OSM nodes

In [4]:
# Generate the nodes dataframe [point_x, point_y, node_id, node_x, node_y, d2node]
# with point = a GPX point and node = the corresponding node
# and d2node = the distance from the GPX point to its corresponding node
filename_nodes = 'cache/' + trailname + '_nodes.csv'
if not os.path.isfile(filename_nodes): # The nodes file does not exist, construct it
    nodes = roadmatch.gpx_to_nodes(gpx, gpx_coords, points_per_batch, delta_roads)
    nodes.to_csv(filename_nodes)
else: # The nodes file exists, load it
    nodes = pd.read_csv(filename_nodes,index_col=0,
                            dtype={'point_x':float, 'point_y':float,
                                   'node_id':int,
                                   'node_x':float, 'node_y':float,
                                   'd2node':float})

Handling batch #0/22 spanning GPX points 0 through 99
   Handling GPX point 99 of 99...
Handling batch #1/22 spanning GPX points 100 through 199
   Handling GPX point 99 of 99...
Handling batch #2/22 spanning GPX points 200 through 299
   Handling GPX point 99 of 99...
Handling batch #3/22 spanning GPX points 300 through 399
   Handling GPX point 99 of 99...
Handling batch #4/22 spanning GPX points 400 through 499
   Handling GPX point 99 of 99...
Handling batch #5/22 spanning GPX points 500 through 599
   Handling GPX point 99 of 99...
Handling batch #6/22 spanning GPX points 600 through 699
   Handling GPX point 99 of 99...
Handling batch #7/22 spanning GPX points 700 through 799
   Handling GPX point 99 of 99...
Handling batch #8/22 spanning GPX points 800 through 899
   Handling GPX point 99 of 99...
Handling batch #9/22 spanning GPX points 900 through 999
   Handling GPX point 99 of 99...
Handling batch #10/22 spanning GPX points 1000 through 1099
   Handling GPX point 99 of 99...

In [8]:
nodes.loc[1300]['node_id']

3009595183.0

In [9]:
# Generate the pieces dataframe [node0, node1, gpx0, gpx1]
# it cuts up the GPX trail into pieces on which individual pathfinding can be done
filename_pieces = 'cache/' + trailname + '_pieces.csv'
if not os.path.isfile(filename_pieces): # The pieces file does not exist, construct it
    pieces = roadmatch.nodes_to_pieces(nodes)
    pieces.to_csv(filename_pieces)
else:
    pieces = pd.read_csv(filename_pieces,index_col=0,
                         dtype={'node0':int,'node1':int,
                                'gpx0':int,'gpx1':int})

In [10]:
pieces

,node0,node1,gpx0,gpx1
0,318732866,4626818866,0,2
1,4626818866,4626818843,2,25
2,4626818843,6617775862,25,28
3,6617775862,318734760,28,40
4,318734760,6110424021,40,47
...,...,...,...,...
626,705827064,2431140715,2263,2264
627,2431140715,705827061,2264,2265
628,705827061,1889821173,2265,2266
629,1889821173,2021911295,2266,2267


In [11]:
npaths = 5

# Generate the segments dataframe [x0,y0,x1,y1,d_cart,d_osm,highway,surface,tracktype]
filename_segments = 'cache/' + trailname + '_segments.csv'
if not os.path.isfile(filename_segments): # The pieces file does not exist, construct it
    segments = roadmatch.pieces_to_segments(gpx,nodes,points_per_batch,delta_roads,pieces,npaths)
    segments.to_csv(filename_segments)
else:
    segments = pd.read_csv(filename_segments,index_col=0,
                         dtype={'x0':float,'y0':float,
                                'x1':float,'y1':float,
                               'd_cart':float,'d_osm':float,
                                'highway':str,'surface':str,'tracktype':str})

Handling piece 19 of 631, spanning GPX points 104 through 109
Handling piece 49 of 631, spanning GPX points 205 through 206
Handling piece 82 of 631, spanning GPX points 303 through 313
Handling piece 124 of 631, spanning GPX points 405 through 406
Handling piece 161 of 631, spanning GPX points 506 through 510
Handling piece 181 of 631, spanning GPX points 610 through 614
Handling piece 199 of 631, spanning GPX points 705 through 712
Handling piece 229 of 631, spanning GPX points 801 through 806
Handling piece 272 of 631, spanning GPX points 907 through 914
Handling piece 306 of 631, spanning GPX points 1001 through 1005
Handling piece 331 of 631, spanning GPX points 1102 through 1104
Handling piece 363 of 631, spanning GPX points 1210 through 1214
Handling piece 381 of 631, spanning GPX points 1301 through 1303
Handling piece 424 of 631, spanning GPX points 1403 through 1410
Handling piece 449 of 631, spanning GPX points 1517 through 1525
Handling piece 466 of 631, spanning GPX points

In [11]:
segments

,point_x,point_y,node_id,node_x,node_y,d2node
0,3.28378,51.24877,318732866,3.283777,51.248771,3.228002e-06
1,3.28415,51.24843,4626818866,3.284196,51.248409,5.084880e-05
2,3.28420,51.24840,4626818866,3.284196,51.248409,9.879271e-06
3,3.28435,51.24843,4626818866,3.284196,51.248409,1.550019e-04
4,3.28442,51.24847,4626818866,3.284196,51.248409,2.317188e-04
...,...,...,...,...,...,...
2265,2.91721,50.82261,705827061,2.917207,50.822602,8.475848e-06
2266,2.91876,50.82151,1889821173,2.918760,50.821511,9.000000e-07
2267,2.91778,50.82097,2021911295,2.917764,50.820979,1.802942e-05
2268,2.92010,50.81924,2021911297,2.920114,50.819238,1.462395e-05


In [13]:
# Map setup
mid = int(len(gpx_coords)/2)
chart = folium.Map(location=gpx_coords[mid], zoom_start=10, tiles="OpenStreetMap")

# Draw the GPX route
newline = folium.PolyLine(locations=gpx_coords, weight=2, color='red')
newline.add_to(chart)

# Draw the matched route
# newline = folium.PolyLine(locations=matchedcoords, weight=2, color='black')
# newline.add_to(chart)

# Draw node mappings
for idx, row in nodes.iterrows():
    row
    point = [row['point_y'],row['point_x']]
    newmarker = folium.CircleMarker(location=point,radius=2,color='red',popup={idx})
    newmarker.add_to(chart)
    node = [row['node_y'],row['node_x']]
    newmarker = folium.CircleMarker(location=node,radius=2,color='blue',popup={row['node_id']})
    newmarker.add_to(chart)
    newline = folium.PolyLine(locations=[point,node], weight=2, color='black')
    newline.add_to(chart)

# Render the map
filepath = f"cache/temp11.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

## Debugging why a wrong node still is returned

In [58]:
n_trail = len(gpx) # Number of GPX points in the trail
n_batch = int(np.ceil(gpx.shape[0]/points_per_batch)) # Number of batches to be run
nodes = pd.DataFrame() # ID of node matched to each GPX point

## --- Loop over all sections of the trail
b = 13

# Define the range of GPX points to process in the current batch
n1 = b*points_per_batch # First point of this batch
n2 = min(n1 + points_per_batch, n_trail) # Last point of this batch (clipped)

print(f'Handling batch #{b}/{n_batch-1} spanning GPX points {n1} through {n2-1}')

gpx_section = gpx.loc[n1:n2] # Select that range of GPX points
section_coords = gpx_coords[n1:n2] # Convert the points into a list of [lat, lon] pairs

lat_min, lat_max, lon_min, lon_max = gr_mapmatch.get_bbox(gpx_section,delta_roads) # Calculate the bounding box
# lat_min = 51.005359999999996 
# lat_max = 51.08834 
# lon_min = 2.8570900000000004 
# lon_max = 2.90



network = gr_mapmatch.get_osm_network(lat_min, lat_max, lon_min, lon_max) # Download the street network from OSM
new_nodes = gr_mapmatch.match_nodes_vec(network,section_coords) # Calculate corresponding node for each GPX point

# Calculate distance between each GPX point and its corresponding node, in this batch
dvec = []
for j in range(len(section_coords)):
    point_id = n1 + j
    point_y = section_coords[j][0]
    point_x = section_coords[j][1]
    node_id = new_nodes[j]
    node = network['points'].loc[node_id]
    node_x = node['x']
    node_y = node['y']
    d2node = gr_mapmatch.get_dist(section_coords[j],[node_y,node_x]) # distance from gpx point to matching node
    newrow = pd.DataFrame({'point_x':point_x, 'point_y':point_y,
                           'node_id':node_id,
                           'node_x':node_x, 'node_y':node_y,
                           'd2node':d2node},index={point_id})
    nodes = pd.concat([nodes,newrow])

Handling batch #13/22 spanning GPX points 1300 through 1399
   Handling GPX point 99 of 99...


In [59]:
# node_pts

# def gpx2points(gpx):
    
x = network['points']['x'].values.tolist()
y = network['points']['y'].values.tolist()
temp = list(zip(x,y))
node_pts = [[point[1],point[0]] for point in temp]

In [60]:
network['points'].loc[3027152111]

y                                  51.037733
x                                   2.896468
street_count                               1
highway                                  NaN
geometry        POINT (2.8964679 51.0377333)
Name: 3027152111, dtype: object

In [68]:
pt = gpx_coords[1300]
nn = ox.distance.nearest_edges(network['graph'],pt[1],pt[0])

In [69]:
nearest_edges
network['edges'].loc[(nn[0],nn[1])]

,osmid,name,highway,surface,oneway,reversed,length,geometry,bridge,tracktype,tunnel
key,,,,,,,,,,,
0,191431401,Boterdijk,track,grass_paver,False,True,1505.986,"LINESTRING (2.87592 51.04124, 2.87601 51.04115...",NaN,grade4,NaN


In [61]:

# Map setup
mid = int(len(gpx_coords)/2)
chart = folium.Map(location=gpx_coords[mid], zoom_start=10, tiles="OpenStreetMap")

for pt in node_pts:
    newmarker = folium.CircleMarker(location=pt,radius=3,color='black')
    newmarker.add_to(chart)

for idx, row in nodes.iterrows():
    point = [row['point_y'],row['point_x']]
    node = [row['node_y'],row['node_x']]
    newmarker = folium.CircleMarker(location=point,radius=2,color='red',popup={idx})
    newmarker.add_to(chart)
    newmarker = folium.CircleMarker(location=node,radius=2,color='blue',popup={row['node_id']})
    newmarker.add_to(chart)
    newline = folium.PolyLine(locations=[point,node], weight=2, color='black')
    newline.add_to(chart)

# # Draw the GPX route
# newline = folium.PolyLine(locations=gpx_coords, weight=2, color='red')
# newline.add_to(chart)

# # Draw the matched route
# newline = folium.PolyLine(locations=matchedcoords, weight=2, color='black')
# newline.add_to(chart)

# Render the map
filepath = f"cache/temp12.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

## Plot the matched route

In [15]:
matchedcoords = gr_plot.get_fullcoords_from_frame(segments)

# Map setup
mid = int(len(gpx_coords)/2)
chart = folium.Map(location=gpx_coords[mid], zoom_start=10, tiles="OpenStreetMap")

# Draw the GPX route
newline = folium.PolyLine(locations=gpx_coords, weight=2, color='red')
newline.add_to(chart)

# Draw the matched route
newline = folium.PolyLine(locations=matchedcoords, weight=2, color='black')
newline.add_to(chart)

# Render the map
filepath = f"cache/temp11.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

## Gathering ROAD data from OSM network

In [4]:
# filename_roads = 'cache/' + trailname + '_roads.csv'

# if not os.path.isfile(filename_roads): # The merged ROADS file does not exist, construct it
    
#     print('Merged ROADS file was not found, merging and saving...')
#     gr_mapmatch.trail2roads(trailname, trail, points_per_batch, delta) # Main batch processor
#     data_roads_raw = gr_utils.merge_roads(trailname, trail, points_per_batch) # Merge the different sections
#     data_roads = gr_mapmatch.remove_repeat_segments(data_roads_raw) # Remove backtracked sections
#     gr_utils.write_roads(trailname, data_roads) # Write the merged ROADS data
# #     print('Saved.')

# else: # The merged ROADS file does exist, just load it
    
#     print('Loading merged ROADS file...')
#     data_roads = gr_utils.read_roads(trailname) # Read the merged sections
#     print('Loaded.')

Loading merged ROADS file...
Loaded.


## Gathering PLACE data from OSM network

In [5]:
# filename_places = 'cache/' + trailname + '_places.csv'

# if not os.path.isfile(filename_places): # The merged PLACES file does not exist
    
#     print('Merged PLACES file was not found, merging and saving...')
#     gr_placematch.roads2places(trailname,data_roads,points_per_batch_places, delta_places, buffersize, tol_area)
#     data_places = gr_utils.merge_places(trailname, data_roads, points_per_batch_places) # Merge the different sections
#     gr_utils.write_places(trailname, data_places) # Write the merged PLACES data
#     print('Saved.')
    
# else: # The merged PLACES file does exist
    
#     print('Loading merged PLACES file...')
#     data_places = gr_utils.read_places(trailname) # Read the merged PLACES file
#     print('Loaded.')

Merged PLACES file was not found, merging and saving...
Handling batch 0 of 41 that covers road segments 0 through 99...
There are 77 uncorrected landuse places
There are 2 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 1 of 41 that covers road segments 100 through 199...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 113 uncorrected landuse places
There are 3 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 2 of 41 that covers road segments 200 through 299...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 370 uncorrected landuse places


...

   Writing outputs to file...
   Finished this batch.
Handling batch 40 of 41 that covers road segments 4000 through 4099...
There are 473 uncorrected landuse places


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


There are 3 corrected landuse places
   Writing outputs to file...
   Finished this batch.
Handling batch 41 of 41 that covers road segments 4100 through 4168...
There are 173 uncorrected landuse places
There are 2 corrected landuse places
   Writing outputs to file...
   Finished this batch.


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


Saved.


## Evaluating PROCESSED properties

In [18]:
# filename_processed = 'data_output/' + trailname + '_processed.csv'

# if not os.path.isfile(filename_processed): # The PROCESSED file does not exist
    
#     print('The PROCESSED file was not found, processing and saving...')
#     data = gr_process.places2processed(data_places,tol_d,types_slow,types_heavy,tracktype_p0,tracktype_p1,tracktype_p2,surface_p0,surface_p1,highway_p1) # Determine traffic/development/paved status & GR types
#     gr_utils.write_processed(trailname, data) # Write the processed data
#     print('Saved.')
    
# else: # The PROCESSED file does exist
    
#     print('The PROCESSED file already exists, so nothing needs to be done.')

The PROCESSED file was not found, processing and saving...
Saved.
